In [1]:
!nvidia-smi

Fri Dec  6 07:15:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip -q install transformers pip==23.1 omegaconf==2.0.6 --use-deprecated=legacy-resolver fairseq vncorenlp

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!rm -rf /kaggle/working/PhoBERT_base_v2_transformers
# !rm -rf /kaggle/working/vncorenlp

In [4]:
# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2024-12-06 07:17:16--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: 'VnCoreNLP-1.1.1.jar'

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.1s    

2024-12-06 07:17:16 (236 MB/s) - 'VnCoreNLP-1.1.1.jar' saved [27412575/27412575]

--2024-12-06 07:17:17--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting respon

In [5]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

tokenizer.save_pretrained('/kaggle/working/PhoBERT_base_v2_transformers/Tokenizer')
phobert.save_pretrained('/kaggle/working/PhoBERT_base_v2_transformers/Model')

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from torch import nn
import json
import numpy as np
import pickle
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from transformers import *
import torch
import matplotlib.pyplot as plt
import torch.utils.data
import torch.nn.functional as F
import argparse
from transformers.modeling_utils import *
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from vncorenlp import VnCoreNLP

/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.
TAPAS models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version. Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.


In [18]:
train_path = '/kaggle/input/data-sentiment-1/TGDD.csv'
dict_path = "/kaggle/working/PhoBERT_base_v2_transformers/Tokenizer/vocab.txt"
config_path = "/kaggle/working/PhoBERT_base_v2_transformers/Model/config.json"
rdrsegmenter_path = "/kaggle/working/vncorenlp/VnCoreNLP-1.1.1.jar"
pretrained_path = '/kaggle/working/PhoBERT_base_v2_transformers/Model/model.safetensors'
max_sequence_length =  200
batch_size = 32
accumulation_steps = 5
epochs = 20
fold = 4
seed = 69
lr = 3e-5
ckpt_path = '/kaggle/working/'
bpe_codes = "/kaggle/working/PhoBERT_base_v2_transformers/Tokenizer/bpe.codes"

In [19]:
def seed_everything(SEED):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

In [20]:
!pip install fastBPE

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp310-cp310-linux_x86_64.whl size=74238 sha256=481561f77e20fda432f7b98d77be3a5ffc25900a46874f78340ad1b7ac888b3d
  Stored in directory: /root/.cache/pip/wheels/13/5d/b9/4b8897941ebc9e8c6cc3f3ffd3ea5115731754269205098754
Successfully built fastBPE


In [21]:
parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes',
    default="/kaggle/working/PhoBERT_base_v2_transformers/Tokenizer/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)

args, unknown = parser.parse_known_args()
bpe = fastBPE(args)
rdrsegmenter = VnCoreNLP("/kaggle/working/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

seed_everything(69)

Loading codes from /kaggle/working/PhoBERT_base_v2_transformers/Tokenizer/bpe.codes ...
Read 64000 codes from the codes file.


In [22]:
import torch
from torch import nn
from transformers import *

class RobertaForAIViVN(BertPreTrainedModel):
   config_class = RobertaConfig
   base_model_prefix = "roberta"
   def __init__(self, config):
       super(RobertaForAIViVN, self).__init__(config)
       self.num_labels = config.num_labels
       self.roberta = RobertaModel(config)
       self.qa_outputs = nn.Linear(4*config.hidden_size, self.num_labels)

       self.init_weights()

   def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):

       outputs = self.roberta(input_ids,
                            attention_mask=attention_mask,
#                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)
       cls_output = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
       logits = self.qa_outputs(cls_output)
       return logits

In [23]:
# Load model
config = RobertaConfig.from_pretrained(
    "/kaggle/working/PhoBERT_base_v2_transformers/Model/config.json",
    from_tf=False,
    num_labels=3,
    output_hidden_states=True,
    attn_implementation="eager"
)
model = RobertaForAIViVN(config)

loading configuration file /kaggle/working/PhoBERT_base_v2_transformers/Model/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base-v2",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "PhobertTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.45.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vo

In [24]:
model.cuda()

RobertaForAIViVN(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [25]:
if torch.cuda.device_count():
    print(f"Training using {torch.cuda.device_count()} gpus")
    model_bert = nn.DataParallel(model)
    tsfm = model_bert.module.roberta
else:
    tsfm = model.roberta

Training using 1 gpus


In [27]:
# Load the dictionary
vocab = Dictionary()
vocab.add_from_file(dict_path)
len(vocab)

64000

In [28]:
train_df = pd.read_csv(train_path, sep=',').fillna("###")

In [29]:
label_encoder = LabelEncoder()
label_encoder.fit(train_df['Sentiment'])
train_df['Sentiment'] = label_encoder.transform(train_df['Sentiment'])

In [30]:
train_df["Sentiment"]

0       0
1       0
2       0
3       0
4       0
       ..
5619    2
5620    2
5621    2
5622    2
5623    2
Name: Sentiment, Length: 5624, dtype: int64

In [31]:
def convert_lines(series, vocab, bpe, max_sequence_length):
    num_samples = len(series)
    outputs = np.zeros((num_samples, max_sequence_length))

    cls_id = 0
    eos_id = 2
    pad_id = 1

    for idx in tqdm(range(num_samples), total=num_samples):
        text = series.iloc[idx]
        subwords = bpe.encode('<s> ' + text + ' </s>')
        input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
        if len(input_ids) > max_sequence_length:
            input_ids = input_ids[:max_sequence_length]
            input_ids[-1] = eos_id
        else:
            input_ids = input_ids + [pad_id] * (max_sequence_length - len(input_ids))
        outputs[idx, :] = np.array(input_ids)

    return outputs

In [32]:
from tqdm import tqdm
tqdm.pandas()

# Chuyển đổi cột 'comment' sang dạng chuỗi
train_df['Cleaned_Review'] = train_df['Cleaned_Review'].astype(str)

train_df["Cleaned_Review"] = train_df['Cleaned_Review'].progress_apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.tokenize(x)]))

y = train_df['Sentiment'].values

X_train = convert_lines(train_df["Cleaned_Review"], vocab, bpe, max_sequence_length)

100%|██████████| 5624/5624 [00:01<00:00, 3517.52it/s]


In [33]:
y[0]

0

In [37]:
# X_train[0]

In [38]:
def preprocess_data(X_train, y):
    # Kiểm tra và chuyển đổi dữ liệu trong X_train
    if isinstance(X_train, np.ndarray) and X_train.dtype == np.object_:
        X_train = np.array([np.array(x, dtype=np.int64) for x in X_train])
    if isinstance(y, np.ndarray) and y.dtype == np.object_:
        y = np.array(y, dtype=np.int64)
    return X_train, y

X_train, y = preprocess_data(X_train, y)

In [39]:
# Nhóm các tham số để áp dụng weight decay khác nhau
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
        'weight_decay': 0.01  # Giảm xuống để tránh overfitting
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
        'weight_decay': 0.0
    }
]

# Sử dụng AdamW với learning rate động
optimizer = AdamW(
    optimizer_grouped_parameters, 
    lr=lr,  # Giảm learning rate nếu cần
    eps=1e-8,  # Thêm eps để tránh chia cho zero
    correct_bias=False
)

# Kết hợp learning rate scheduler với warmup
num_train_optimization_steps = int(epochs * len(train_df) / batch_size / accumulation_steps)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=int(0.05 * num_train_optimization_steps),  # Giảm warmup steps
    num_training_steps=num_train_optimization_steps
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [40]:
# Thêm early stopping để ngăn chặn overfitting
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_score is None:
            self.best_score = val_loss
        elif val_loss > self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_loss
            self.counter = 0

In [41]:
def softmax(x):
    """Tính toán hàm softmax cho mảng đầu vào x."""
    e_x = np.exp(x - np.max(x))  # Sử dụng np.max(x) để cải thiện sự ổn định số học
    return e_x / e_x.sum(axis=1, keepdims=True)

In [ ]:
if not os.path.exists(ckpt_path):
    os.mkdir(ckpt_path)

early_stopper = EarlyStopping(patience=5, min_delta=0.001)
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(X_train, y))
for fold, (train_idx, val_idx) in enumerate(splits):
    print("Training for fold {}".format(fold))
    best_score = 0
    if fold != fold:
        continue

    # Create train and validation datasets
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train[train_idx],dtype=torch.long), torch.tensor(y[train_idx],dtype=torch.long))
    valid_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train[val_idx],dtype=torch.long), torch.tensor(y[val_idx],dtype=torch.long))
    tq = tqdm(range(epochs + 1))

    # Đóng băng (freeze) các tham số của một số lớp con của mô hình
    for child in tsfm.children():
        for param in child.parameters():
            if not param.requires_grad:
                print("whoopsies")
            param.requires_grad = False
    frozen = True


    # Điều chỉnh trạng thái huấn luyện của mô hình trong mỗi epoch
    for epoch in tq:
        if epoch > 0 and frozen:
            for child in tsfm.children():
                for param in child.parameters():
                    param.requires_grad = True
            frozen = False
            torch.cuda.empty_cache()  # Chỉ giải phóng bộ nhớ tạm

            early_stopper(score)
            if early_stopper.early_stop:
                print(f"Stopping training for fold {fold}")
                break

            if score >= best_score:
                torch.save(model.state_dict(), os.path.join(ckpt_path, f"model_{fold}.bin"))
                best_score = score

        val_preds = None
        # Khởi tạo DataLoader cho tập train và valid
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
        avg_loss = 0.
        avg_accuracy = 0.
        # Đặt gradient của các tham số mô hình về 0.0
        optimizer.zero_grad()


        # Thực hiện huấn luyện mô hình, tính toán mất mát, cập nhật trọng số và theo dõi tiến trình huấn luyện.

        criterion = nn.CrossEntropyLoss()
        pbar = tqdm(enumerate(train_loader),total=len(train_loader),leave=False)
        for i, (x_batch, y_batch) in pbar:
          model.train()
          y_pred = model(x_batch.cuda(), attention_mask=(x_batch > 0).cuda())

          y_batch = y_batch.cuda()  # Nhãn thực tế

          # Tính toán mất mát
          loss = criterion(y_pred, y_batch)
          loss.backward()

          if i % accumulation_steps == 0 or i == len(pbar) - 1:
              optimizer.step()
              optimizer.zero_grad()
              if not frozen:
                scheduler.step()
              else:
                if 'scheduler0' in locals():  # Chỉ sử dụng scheduler0 nếu đã được định nghĩa
                    scheduler0.step()
                else:
                    print("Warning: scheduler0 is not defined.")


          lossf = loss.item()
          pbar.set_postfix(loss=lossf)
          avg_loss += loss.item() / len(train_loader)

        # Đánh giá mô hình trên tập validation, thu thập và lưu trữ các dự đoán của mô hình, và áp dụng hàm softmax để chuyển đổi các giá trị logits thành xác suất.
        pbar = tqdm(enumerate(valid_loader), total=len(valid_loader), leave=False)
        for i, (x_batch, y_batch) in pbar:
            model.eval()  # Đặt mô hình ở chế độ đánh giá
            with torch.no_grad():  # Tắt gradient để tiết kiệm bộ nhớ
                y_pred = model(x_batch.cuda(), attention_mask=(x_batch > 0).cuda())

            # Giả sử y_pred có kích thước [batch_size, num_classes]
            y_pred = y_pred.cpu().numpy()  # Chuyển dự đoán về CPU và NumPy
            val_preds = np.atleast_1d(y_pred) if val_preds is None else np.concatenate([val_preds, np.atleast_1d(y_pred)])

        # Áp dụng softmax cho dự đoán
        val_preds = softmax(val_preds)  # Đảm bảo softmax cho lớp cuối cùng

        # Tính điểm F1 và AUC
        y_true = y[val_idx]
        y_pred_labels = np.argmax(val_preds, axis=1)  # Chọn lớp với xác suất cao nhất
        score = f1_score(y_true, y_pred_labels, average='weighted')  # Tính điểm F1 với lựa chọn trung bình

        print(f"\nAUC = {roc_auc_score(y_true, val_preds, multi_class='ovr'):.4f}, F1 score = {score:.4f}")

        # Lưu mô hình nếu điểm F1 tốt nhất hiện tại
        if score >= best_score:
            torch.save(model.state_dict(), os.path.join(ckpt_path, f"model_{fold}.bin"))
            best_score = score

Training for fold 0


  1%|          | 1/141 [00:00<01:28,  1.59it/s, loss=1.19]


  4%|▍         | 6/141 [00:01<00:31,  4.33it/s, loss=1.36]


  8%|▊         | 11/141 [00:02<00:27,  4.77it/s, loss=1.38]


 11%|█▏        | 16/141 [00:03<00:25,  4.84it/s, loss=1.12]

...

100%|██████████| 141/141 [01:23<00:00,  1.90it/s, loss=0.0401]
                                                              
 97%|█████████▋| 35/36 [00:07<00:00,  4.98it/s]
                                               


AUC = 1.0000, F1 score = 0.9982


100%|██████████| 141/141 [01:23<00:00,  1.90it/s, loss=0.0317]
                                                              
 97%|█████████▋| 35/36 [00:07<00:00,  4.97it/s]
                                               


AUC = 1.0000, F1 score = 0.9982


 49%|████▉     | 69/141 [00:41<00:42,  1.69it/s, loss=0.144]

In [ ]:
import torch
torch.save(model.state_dict(), "model_weights.pth")

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


# Đọc dữ liệu test từ tệp CSV
test_df = pd.read_csv('/kaggle/input/test-data/test.csv', sep=',').fillna("###")
label_encoder = LabelEncoder()
label_encoder.fit(test_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

test_df['comment'] = test_df['comment'].astype(str)

test_df["comment"] = test_df['comment'].progress_apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.tokenize(x)]))

y_test = test_df['label'].values

X_test = convert_lines(test_df["comment"], vocab, bpe, max_sequence_length)

In [ ]:
# Chuyển đổi dữ liệu test thành tensor
test_inputs = torch.tensor(X_test, dtype=torch.long)
test_labels = torch.tensor(y_test, dtype=torch.long)

# Tạo dataset và dataloader
test_dataset = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

cofig = RobertaConfig.from_pretrained(
    "/kaggle/working/PhoBERT_base_v2_transformers/Model/config.json",
    from_tf=False,
    num_labels=3,
    output_hidden_states=True,
    attn_implementation="eager"
)
model = RobertaForAIViVN(config)
model.load_state_dict(torch.load('/kaggle/working/model_weights.pth', map_location=torch.device('cpu')))
model.eval()  # Đặt mô hình ở chế độ đánh giá

all_preds = []
all_labels = []


with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids, labels = batch
        outputs = model(input_ids)
        if isinstance(outputs, torch.Tensor):
            logits = outputs
        else:
            logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        print(f'Predictions shape: {predictions.shape}')
        print(f'Labels shape: {labels.shape}')

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Tính toán confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Vẽ confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()